In [25]:
import pandas as pd
pd.set_option('display.max_rows' , 100)

In [26]:
company_name = 'MASTEK'

In [27]:
share_price_in = 1000000;

In [28]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')

In [29]:
start_col = income_statement.columns[1];

In [30]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : 'items'} , inplace=True)

In [31]:
basic_eps_filter = income_statement['items'] == 'Basic EPS'
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [32]:
WASO_filter = (income_statement['items'] == 'Basic Weighted Average Shares Outstanding') | (income_statement['items'] == 'Diluted Weighted Average Shares Outstanding') | (income_statement['items'] == 'Basic WASO') | (income_statement['items'] == 'Diluted WASO')
waso_pos = income_statement[WASO_filter].index

In [33]:
income_statement.fillna(0 , inplace=True)

In [34]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [35]:
def get_share_price_in(price) :
    return round(price / share_price_in,2)

    

In [36]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [37]:
temp_df = temp_df.apply(get_share_price_in)

In [38]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [39]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [40]:
WASO_filter = (income_statement['items'] == 'Basic Weighted Average Shares Outstanding') | (income_statement['items'] == 'Diluted Weighted Average Shares Outstanding') | (income_statement['items'] == 'Basic WASO') | (income_statement['items'] == 'Diluted WASO')

In [41]:
income_statement[WASO_filter].index

Int64Index([76, 77, 82, 83], dtype='int64')

In [42]:
income_statement

,items,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Gross Profit,3581.56,4145.61,2464.21,2444.73,3323.40,4401.70,5189.10,8775.40,11395.90,0.00
1,Total Revenue,9215.66,10122.30,5267.04,5610.34,8172.10,10332.10,10714.80,17218.60,21838.40,23291.10
2,Business Revenue,9095.68,10019.63,5252.66,5594.22,8156.40,10332.10,10699.30,17207.70,21798.60,23251.30
3,Other Revenue,119.98,102.67,14.38,16.12,15.70,0.00,15.50,10.90,39.80,0.00
4,Cost of Revenue,-5634.10,-5976.69,-2802.83,-3165.60,-4848.70,-5930.40,-5525.70,-8443.20,-10442.50,0.00
5,Cost of Goods and Services,-50.04,-39.16,0.00,0.00,0.00,0.00,-47.70,-179.40,-103.10,0.00
6,"Staff Cost, Cost of Revenue",-5584.06,-5937.53,-2802.83,-3165.60,-4848.70,-5930.40,-5478.00,-8263.80,-10339.40,0.00
7,Operating Income/Expenses,-2961.74,-3759.61,-2283.87,-2026.66,-2512.70,-3333.00,-4070.70,-5441.70,-7199.30,-2333.10
8,"Selling, General and Administrative Expenses",-1786.39,-2366.38,-1634.51,-1431.64,-1750.10,-2464.80,-2968.80,-4449.10,-5757.30,0.00
9,Staff Costs,-89.57,-306.28,-58.83,-77.90,0.00,0.00,-434.10,-731.60,-828.40,0.00
